In [6]:
import pandas as pd

In [7]:
import time

In [8]:
pd.set_option('display.max_columns', 100)

empirically, loading data from "signed_predictors_dl_wide.csv" takes 2 mins

loading data from "sp500_op_ret.csv" takes 10 secs

I am using macbook pro with intel i7, 16GB memory

In [9]:
start = time.time()
signed_predictors_dl_wide = pd.read_csv("data/signed_predictors_dl_wide.csv") 
print(time.time() - start)

98.4389419555664


In [11]:
start = time.time()
sp500_op_ret = pd.read_csv("data/sp500_op_ret.csv")
print(time.time() - start)

10.234528064727783


In [12]:
mapping_table = pd.read_csv("data/mapping_table.csv")

In [13]:
sp500_const_list = pd.read_csv("data/sp500_const_list.csv")

In [14]:
sp500_op_ret = sp500_op_ret.merge(mapping_table, on="secid", how="left")

In [15]:
# filter sparse entries

In [16]:
na_freq_dict = dict(
    signed_predictors_dl_wide.isna().sum() / signed_predictors_dl_wide.shape[0])

In [17]:
[key for key, val in na_freq_dict.items() if val < 0.2]

['permno',
 'yyyymm',
 'Beta',
 'BidAskSpread',
 'Coskewness',
 'DolVol',
 'High52',
 'IdioRisk',
 'IdioVol3F',
 'IdioVolAHT',
 'MaxRet',
 'PriceDelayRsq',
 'PriceDelaySlope',
 'PriceDelayTstat',
 'ReturnSkew',
 'ReturnSkew3F',
 'VolMkt',
 'zerotrade',
 'zerotradeAlt1']

In [23]:
import datetime

In [18]:
signed_predictors_dl_wide = signed_predictors_dl_wide[
    [key for key, val in na_freq_dict.items() if val < 0.2]
]

In [32]:
signed_predictors_dl_wide["yyyymm"].iloc[:3].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m'))

0   1986-01-01
1   1986-02-01
2   1986-03-01
Name: yyyymm, dtype: datetime64[ns]

In [38]:
signed_predictors_dl_wide["date"] = signed_predictors_dl_wide["yyyymm"].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m'))

In [39]:
sp500_op_ret["date2"] = sp500_op_ret.date.apply(
    lambda x: datetime.datetime.strptime(str(x)[:7], '%Y-%m'))

In [48]:
df_merged = sp500_op_ret.merge(
    signed_predictors_dl_wide, 
    left_on=["permno", "date2"], right_on=["permno", "date"],
    how="left"
)

In [51]:
df_merged.to_csv("data/option_with_nonsparse_features.csv")

In [42]:
len(set(sp500_op_ret["date2"]) - set(signed_predictors_dl_wide["date"]))

0

In [41]:
len(set(signed_predictors_dl_wide["date"]))

1159

In [45]:
df1 = pd.DataFrame({
    "f1": [1, 1, 1],
    "time": [1,2,3]
})

df2 = pd.DataFrame({
    "f1": [1, 1, 1],
    "time": [1, 2, 3],
    "feature": [0,9,8]
})

In [47]:
df1.merge(df2, left_on=["f1", "time"], right_on = ["f1", "time"], how="left")

,f1,time,feature
0,1,1,0
1,1,2,9
2,1,3,8


In [ ]:
df1.merge(df2, on="f1", how="left")

,f1,time_x,time_y,feature
0,1,1,1,0
1,1,1,2,9
2,1,1,3,8
3,1,2,1,0
4,1,2,2,9
5,1,2,3,8
6,1,3,1,0
7,1,3,2,9
8,1,3,3,8


In [15]:
sp500_op_permno = set(sp500_op_ret["permno"])

In [ ]:
sp500_op_ret.merge(
    signed_predictors_dl_wide[signed_predictors_dl_wide["permno"].isin(sp500_op_permno)],
    on="permno", how="left"
)

In [14]:
len(set(signed_predictors_dl_wide["permno"]))

34424

In [12]:
# merge on permno 

In [ ]:
sp500_op_ret.merge(signed_predictors_dl_wide, on="permno", how="inner")

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [43]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [54]:
from sklearn import linear_model